In [1]:
import utils

import pandas as pd

In [2]:
cost = pd.read_csv("data/norm-cost.csv",index_col=0)
# bonds from solver
solv_bonds = pd.read_csv("data/all-complete.csv",sep=";")[["instance_number","int_objective_bound"]].groupby("instance_number").max().reset_index()
solv_bonds = pd.merge(solv_bonds,cost)
solv_bonds["norm_objective_bound"] = solv_bonds["int_objective_bound"]/solv_bonds["normalization_constant"]


# select bounds from theory
the_bounds = pd.read_csv("data/theoretical-lower-bounds.csv")\
    .rename(columns={
        "normalised objective bound":"norm_objective_bound",
    })[["instance_number","norm_objective_bound"]]

best_bonds = pd.concat([solv_bonds,the_bounds])[["instance_number","norm_objective_bound"]].groupby("instance_number").max().reset_index()

In [3]:
# best exact methods
best_exact = pd.read_csv("data/all-complete.csv",sep=";").query("solutionCosts >= 0.")[["instance_number","solutionCosts"]].groupby("instance_number").min().reset_index()

In [4]:
df = pd.merge(
    best_exact,
    best_bonds
)
df["gap"] = df.apply(lambda row: utils.perc_gap(row["solutionCosts"],row["norm_objective_bound"]), axis=1)
df["gap_type"] = df.apply(lambda row: utils.gap_type(row["gap"]),axis=1)
df["nJobs"] = df.apply(lambda row: utils.get_n_jobs(row["instance_number"]), axis=1)
df["size"] = df.apply(lambda row: utils.get_size(instance_number=row["instance_number"]), axis=1)
df.head()

,instance_number,solutionCosts,norm_objective_bound,gap,gap_type,nJobs,size
0,1,0.792571,0.792571,0.0,gap = 0%,10 jobs,small
1,2,0.977937,0.977937,0.0,gap = 0%,10 jobs,small
2,3,0.225556,0.225556,-0.0,gap = 0%,10 jobs,small
3,4,0.492381,0.492381,-0.0,gap = 0%,10 jobs,small
4,5,0.308648,0.308648,0.0,gap = 0%,10 jobs,small


In [5]:
df[["instance_number","gap_type","size"]].groupby(["size","gap_type"]).count().reset_index()

,size,gap_type,instance_number
0,large,0% < gap ≤ 1%,18
1,large,1% < gap ≤ 5%,3
2,large,gap > 5%,19
3,medium,0% < gap ≤ 1%,18
4,medium,1% < gap ≤ 5%,1
5,medium,gap = 0%,4
6,medium,gap > 5%,17
7,small,gap = 0%,37
8,small,gap > 5%,3


In [6]:
df.query("gap <= 0.0").shape

(41, 7)

In [7]:
df.query("gap <= 0.0").describe()

,instance_number,solutionCosts,norm_objective_bound,gap
count,41.000000,41.000000,41.000000,41.0
mean,22.951220,0.739153,0.739153,0.0
std,15.620421,0.271061,0.271061,0.0
min,1.000000,0.136706,0.136706,-0.0
25%,11.000000,0.534539,0.534539,0.0
50%,21.000000,0.875887,0.875887,0.0
75%,32.000000,0.970615,0.970615,0.0
max,75.000000,0.993798,0.993798,0.0


In [8]:
df.query("gap > 0.0").describe()

,instance_number,solutionCosts,norm_objective_bound,gap
count,79.000000,79.000000,79.000000,79.000000
mean,79.987342,0.849786,0.734677,15.396835
std,24.396035,0.172080,0.264938,20.070174
min,24.000000,0.298674,0.182654,0.020000
25%,60.500000,0.814505,0.487831,0.435000
50%,81.000000,0.924286,0.859052,3.970000
75%,100.500000,0.971565,0.966491,28.035000
max,120.000000,0.984676,0.984493,78.620000


In [15]:
100*df.query("gap <= 1.01 and size in ['medium','large']").shape[0]/80

50.0

In [13]:
18+18+4

40